In [1]:
import numpy as np
import pandas as pd
import math
import sys, os
#import seaborn as sns
#from scipy import stats
#from pathlib import Path
#import matplotlib.pyplot as plt
#from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
#from sklearn.metrics import mean_squared_error
#from sklearn.metrics import mean_squared_error
#from sklearn.preprocessing import MinMaxScaler
#from sklearn.datasets import fetch_california_housing
#import math
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
import optuna
from optuna.samplers import TPESampler
import lightgbm as lgbm
#from xgboost import XGBRegressor
#import xgboost as xgb
#from catboost import CatBoostRegressor
#from lightgbm.sklearn import LGBMRegressor
#from sklearn.ensemble import RandomForestRegressor
import pickle
from sklearn.metrics import roc_curve, auc

In [2]:
sys.path.append(os.path.realpath('..'))

In [3]:
train_df = pd.read_csv(r"..\data\train.csv", index_col=0)
test_df = pd.read_csv(r"..\data\test.csv", index_col=0)
sample_sub = pd.read_csv(r"..\data\sample_submission.csv", index_col=0)

In [4]:
del train_df['Over18']
del train_df['EmployeeCount']
del train_df['StandardHours']

In [5]:
columns_to_vectorize = ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'OverTime']
for vector_target in columns_to_vectorize:
    print(vector_target)
    vectorizer = CountVectorizer()
    vectorizer.fit_transform(train_df[vector_target])
    train_df[f'{vector_target}_v'] = vectorizer.transform(train_df[vector_target]).toarray().argmax(axis=1)[:,None]
    vectorizer.fit_transform(test_df[vector_target])
    test_df[f'{vector_target}_v'] = vectorizer.transform(test_df[vector_target]).toarray().argmax(axis=1)[:,None]

BusinessTravel
Department
EducationField
Gender
JobRole
MaritalStatus
OverTime


In [6]:
features = ['Age', 'DailyRate', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction','HourlyRate', 'JobInvolvement', 'JobLevel', 'JobSatisfaction', 
       'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel', 'TotalWorkingYears',
       'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager',
        'BusinessTravel_v', 'Department_v', 'EducationField_v','Gender_v', 'JobRole_v', 'MaritalStatus_v', 'OverTime_v']
target = ['Attrition']

In [7]:
scaler = MinMaxScaler().fit(train_df[features])
X = scaler.transform(train_df[features])
X_test = scaler.transform(test_df[features])

scaler = MinMaxScaler().fit(train_df[target])
Y = scaler.transform(train_df[target])

In [8]:
def lgbm_objective(trial):
    # Split the train data for each trial.
    X_train, X_valid, y_train, y_valid = train_test_split(X, Y, stratify=Y, test_size=0.2)

    param_grid = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.5),
        'max_depth': trial.suggest_int('max_depth', 100, 1000), 
        'num_leaves': trial.suggest_int('num_leaves', 100, 10000),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1),
        'subsample': trial.suggest_float('subsample', 0.1, 1),
        'subsample_freq': trial.suggest_int('subsample_freq', 1, 10), 
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 1000), 
        'reg_lambda': trial.suggest_int('reg_lambda', 1, 100), 
        'n_estimators': trial.suggest_int('n_estimators', 10, 100000), 
    } 
    
    clf = lgbm.LGBMRegressor(**param_grid, metric='AUC', random_state=1)
    
    clf.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], callbacks=[lgbm.early_stopping(100, verbose=True)], verbose=False)
    preds = clf.predict(X_valid)

    fpr, tpr, _ = roc_curve(y_valid, preds)
    roc_auc = auc(fpr, tpr)

    lgbm_ranks[roc_auc] = clf

    return roc_auc

In [9]:
lgbm_ranks = {}

train_time = 1 * 60 * 1
study = optuna.create_study(direction='maximize', sampler=TPESampler(), study_name='LGBMRegressor')
study.optimize(lgbm_objective, timeout=train_time)

[I 2023-01-16 19:49:10,594] A new study created in memory with name: LGBMRegressor
D:\source\repos\venv\Python310\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\source\repos\venv\Python310\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
D:\source\repos\venv\Python310\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
[I 2023-01-16 19:49:10,617] Trial 0 finished with value: 0.5 and parameters: {'learning_rate': 0.06280018175797787, 'max_dep

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.5
Training until validation scores don't improve for 100 rounds


D:\source\repos\venv\Python310\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\source\repos\venv\Python310\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
D:\source\repos\venv\Python310\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
[I 2023-01-16 19:49:10,688] Trial 1 finished with value: 0.5 and parameters: {'learning_rate': 0.039429809713922216, 'max_depth': 105, 'num_leaves': 6367, 'colsample_bytree': 0.5840341541776318, 'subsample':

Early stopping, best iteration is:
[1]	valid_0's auc: 0.5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.5
Training until validation scores don't improve for 100 rounds


[I 2023-01-16 19:49:10,912] Trial 3 finished with value: 0.8008037225042302 and parameters: {'learning_rate': 0.08274423851778089, 'max_depth': 238, 'num_leaves': 1791, 'colsample_bytree': 0.6816194816902676, 'subsample': 0.6245970447761168, 'subsample_freq': 4, 'min_child_samples': 21, 'reg_lambda': 25, 'n_estimators': 12561}. Best is trial 3 with value: 0.8008037225042302.


...

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[129]	valid_0's auc: 0.838071


[I 2023-01-16 19:54:10,082] Trial 712 finished with value: 0.8380710659898477 and parameters: {'learning_rate': 0.09755121711138513, 'max_depth': 211, 'num_leaves': 9554, 'colsample_bytree': 0.5004908940865591, 'subsample': 0.6104817788383793, 'subsample_freq': 2, 'min_child_samples': 104, 'reg_lambda': 56, 'n_estimators': 84457}. Best is trial 388 with value: 0.8805414551607444.
D:\source\repos\venv\Python310\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\source\repos\venv\Python310\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
D

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[119]	valid_0's auc: 0.828299


[I 2023-01-16 19:54:10,560] Trial 713 finished with value: 0.8282994923857868 and parameters: {'learning_rate': 0.09233217245064775, 'max_depth': 407, 'num_leaves': 9298, 'colsample_bytree': 0.3157736386221217, 'subsample': 0.8070384399943937, 'subsample_freq': 2, 'min_child_samples': 69, 'reg_lambda': 40, 'n_estimators': 83028}. Best is trial 388 with value: 0.8805414551607444.
D:\source\repos\venv\Python310\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\source\repos\venv\Python310\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
D:

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[202]	valid_0's auc: 0.796024


In [10]:
LGBM_ranks_bytes = pickle.dumps(lgbm_ranks)
with open("rendered_data/lgbm_clsfs_bytes.hex", "wb") as binary_file:
    binary_file.write(LGBM_ranks_bytes)

In [11]:
percent = math.ceil(len(lgbm_ranks.keys()) * .3)
top = sorted(list(lgbm_ranks.keys()))[-percent:]

train_preds = []
test_preds = []
for key in tqdm(top):
    train_preds.append(lgbm_ranks[key].predict(X))
    test_preds.append(lgbm_ranks[key].predict(X_test))
    
train_final_preds = np.stack(train_preds).mean(0)
test_final_preds = np.stack(test_preds).mean(0)

100%|█████████████████████████████████████████████████████████████████████████████████| 64/64 [00:00<00:00, 412.34it/s]


In [14]:
LGBM_train_submission = pd.DataFrame(data={'id': train_df.index, target[0]: train_final_preds})
LGBM_train_submission.to_csv(fr'rendered_data/LGBM_train_submission.csv', index=False)

LGBM_test_submission = pd.DataFrame(data={'id': test_df.index, target[0]: test_final_preds})
LGBM_test_submission.to_csv(fr'rendered_data/LGBM_test_submission.csv', index=False)